In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\Aryan Aarav\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [3]:
tokens = tokenizer.encode('Wow! well done.', return_tensors = 'pt')

In [4]:
tokens

tensor([[  101, 94608,   106, 11327, 19123,   119,   102]])

In [5]:
result = model(tokens)

In [6]:
tokenizer.decode(tokens[0])

'[CLS] wow! well done. [SEP]'

In [7]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.2826, -2.6213, -0.8458,  1.6097,  3.3583]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
result.logits

tensor([[-2.2826, -2.6213, -0.8458,  1.6097,  3.3583]],
       grad_fn=<AddmmBackward0>)

In [9]:
int(torch.argmax(result.logits)) + 1

5

In [10]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')

In [16]:
regex = re.compile('.*comments*.')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [17]:
reviews[0]

'The food is fresh and tasty.  The scallop ceviche started the lunch. The scallops were tender with a great acidity and use of mango and peppers. The steak was tender and I got the hint of tequila in the sauce. I enjoyed a watermelon salad that complimented the the steak. The portions are good, but a stretch if you are sharing. My only down point is the service. They really only showed up to present my next plate and never checked to see if I wanted another drink (which I did).Enjoyed the food.'

In [19]:
import pandas as pd
import numpy as np

In [20]:
df = pd.DataFrame(np.array(reviews), columns = ['reviews'])

In [21]:
df

,reviews
0,The food is fresh and tasty. The scallop cevi...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...
5,Don't come here expecting legit Mexican food b...
6,Out of all the restaurants that I tried in Syd...
7,"Great atmosphere, attentive service, solid mar..."
8,We came here on a Thursday night @ 5pm and by ...
9,Have been here twice and have absolutely loved...


In [26]:
# creating the sentiment function

def sentiment_values(review):
    tokens = tokenizer.encode(review, return_tensors = 'pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+ 1

# now we need to loop through each one of the reviews in the dataframe
df['sentiment'] = df['reviews'].apply(lambda x: sentiment_values(x[:]))

In [27]:
df

,reviews,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,Have been here twice and have absolutely loved...,5
